In [1]:
import joblib
import json
import pandas
import warnings
import numpy

import sklearn.svm
import sklearn.neural_network
import sklearn.model_selection
import sklearn.exceptions

warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)

# Instructions

- Read **the train data** from the CSV file and properly set the index


In [2]:
data_train = pandas.read_csv('./data/features.train.csv').set_index('id')
data_train

,workclass_self-emp-not-inc,workclass_private,workclass_self-emp-inc,workclass_local-gov,workclass_federal-gov,workclass_state-gov,workclass_never-worked,workclass_without-pay,marital-status_never-married,marital-status_married-civ-spouse,...,native-country-code_LAO,native-country-code_PER,native-country-code_NLD,age-group,fnlwgt,capitalgain,capitalloss,hoursperweek,education-num,label
id,,,,,,,,,,,,,,,,,,,,,
2103,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,1.395470,-0.328536,6.084073,-0.051360,1.760717,1.0
14649,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,-0.873898,-0.328536,-0.230847,1.095785,-0.553886,0.0
7379,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,-0.592186,-0.328536,-0.230847,2.242930,-0.553886,0.0
24479,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,-1.349595,-0.328536,-0.230847,1.095785,-0.168119,0.0
19532,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,2.041210,-0.328536,-0.230847,-0.051360,-3.254256,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8695,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,-1.405521,-0.328536,-0.230847,-0.051360,-0.168119,0.0
2192,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,-0.821560,1.795856,-0.230847,-0.051360,-0.168119,0.0
8250,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.210247,-0.328536,-0.230847,-0.051360,-0.553886,0.0


In [3]:
search_parameters = {
    'alpha': numpy.arange(0.0025, 0.003, 0.0001),
}

search_parameters = {
    # 'c': [0.2, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]
    'C': numpy.arange(5, 18, 2),
}

In [4]:
model = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes    = (17,8,17), # 17 8 17
    solver                = 'sgd', #sgd
    activation            = 'logistic', #logistic
    alpha                 = 0.00289,# 0.0038 for 17, 8 ,,,, 0.00289 for 17,8,17
    batch_size            = 36, #36 for 17,8,17
    learning_rate         = 'constant',
    learning_rate_init    = 0.12, #0.12
    max_iter              = 1000, #doesnt change
    momentum              = 0.0, 
    nesterovs_momentum    = True, 
    validation_fraction   = 0.2, 
    shuffle               = True,
    random_state          = 0
)

model = sklearn.svm.SVC(
    C= 0, 
    kernel='rbf', 
    shrinking=True, 
    random_state=0
)
model

In [5]:
model = sklearn.model_selection.GridSearchCV(
    estimator=model, 
    param_grid=search_parameters,
    n_jobs=-1,
    verbose=2
)

In [6]:
model.fit(
    data_train.drop(['label'], axis='columns'),
    data_train['label'],
)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(estimator=MLPClassifier(activation='logistic', alpha=0.00289,
                                     batch_size=36,
                                     hidden_layer_sizes=(17, 8, 17),
                                     learning_rate_init=0.12, max_iter=1000,
                                     momentum=0.0, random_state=0, solver='sgd',
                                     validation_fraction=0.2),
             n_jobs=-1,
             param_grid={'alpha': array([0.0025, 0.0026, 0.0027, 0.0028, 0.0029])},
             verbose=2)

In [7]:
cv_results = pandas.DataFrame(model.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,50.355817,4.473034,0.016804,0.004363,0.0025,{'alpha': 0.0025},0.790112,0.824561,0.815630,0.810207,0.822329,0.812568,0.012315,2
1,48.889748,4.524195,0.022172,0.005890,0.0026,{'alpha': 0.0026},0.789793,0.824561,0.814992,0.810207,0.822648,0.812440,0.012454,4
2,52.621883,2.866167,0.015902,0.005735,0.0027,{'alpha': 0.0026999999999999997},0.788836,0.824242,0.815311,0.810845,0.822967,0.812440,0.012791,4
3,38.230768,8.482355,0.006384,0.004240,0.0028,{'alpha': 0.0027999999999999995},0.788836,0.824561,0.815630,0.811164,0.822648,0.812568,0.012806,2
4,32.049612,1.503076,0.003789,0.000479,0.0029,{'alpha': 0.0028999999999999994},0.813716,0.824242,0.815311,0.812121,0.822010,0.817480,0.004771,1


In [8]:
cv_results.sort_values(['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,32.049612,1.503076,0.003789,0.000479,0.0029,{'alpha': 0.0028999999999999994},0.813716,0.824242,0.815311,0.812121,0.822010,0.817480,0.004771,1
0,50.355817,4.473034,0.016804,0.004363,0.0025,{'alpha': 0.0025},0.790112,0.824561,0.815630,0.810207,0.822329,0.812568,0.012315,2
3,38.230768,8.482355,0.006384,0.004240,0.0028,{'alpha': 0.0027999999999999995},0.788836,0.824561,0.815630,0.811164,0.822648,0.812568,0.012806,2
1,48.889748,4.524195,0.022172,0.005890,0.0026,{'alpha': 0.0026},0.789793,0.824561,0.814992,0.810207,0.822648,0.812440,0.012454,4
2,52.621883,2.866167,0.015902,0.005735,0.0027,{'alpha': 0.0026999999999999997},0.788836,0.824242,0.815311,0.810845,0.822967,0.812440,0.012791,4


In [9]:
model.best_params_

{'alpha': 0.0028999999999999994}